In [1]:
import argparse
import os
import sys
import time

import numpy as np
import tensorflow as tf

In [2]:
def get_command_arguments():
    """ Read input variables and parse command-line arguments """

    parser = argparse.ArgumentParser(
        description='Train a simple Convolutional Neural Network to classify CIFAR images.',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )

    parser.add_argument('-c', '--classes', type=int, default=10, choices=[10, 20, 100], help='number of classes in dataset')
    parser.add_argument('-p', '--precision', type=str, default='fp32', choices=['bf16', 'fp16', 'fp32', 'fp64'], help='floating-point precision')
    parser.add_argument('-e', '--epochs', type=int, default=42, help='number of training epochs')
    parser.add_argument('-b', '--batch_size', type=int, default=256, help='batch size')

    args = parser.parse_args()
    return args

In [3]:
def create_datasets(classes, dtype):
    """ Create CIFAR training and test datasets """

    # Download training and test image datasets
    if classes == 100:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
    elif classes == 20:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='coarse')
    else: # classes == 10
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Verify training and test image dataset sizes
    assert x_train.shape == (50000, 32, 32, 3)
    assert y_train.shape == (50000, 1)
    assert x_test.shape == (10000, 32, 32, 3)
    assert y_test.shape == (10000, 1)

    # Normalize the 8-bit (3-channel) RGB image pixel data between 0.0
    # and 1.0; also converts datatype from numpy.uint8 to numpy.float64
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    # Convert from NumPy arrays to TensorFlow tensors
    x_train = tf.convert_to_tensor(value=x_train, dtype=dtype, name='x_train')
    y_train = tf.convert_to_tensor(value=y_train, dtype=tf.uint8, name='y_train')
    x_test = tf.convert_to_tensor(value=x_test, dtype=dtype, name='x_test')
    y_test = tf.convert_to_tensor(value=y_test, dtype=tf.uint8, name='y_test')

    # Construct TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

    return train_dataset, test_dataset

In [4]:
def create_model(classes):
    """ Specify and compile the CNN model """

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(classes),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    return model

In [5]:
def train_model(classes, precision, epochs, batch_size):
    """ Train CNN on CIFAR """

    # Create training and test datasets
    train_dataset, test_dataset = create_datasets(classes, dtype=precision)

    # Prepare the datasets for training and evaluation
    train_dataset = train_dataset.cache().shuffle(buffer_size=50000, reshuffle_each_iteration=True).batch(batch_size)
    test_dataset = test_dataset.batch(batch_size)

    # Create model
    model = create_model(classes)

    # Print summary of the model's network architecture
    model.summary()

    # Train the model on the dataset
    model.fit(x=train_dataset, epochs=epochs, verbose=2)

    # Evaluate the model and its accuracy
    model.evaluate(x=test_dataset, verbose=2)

    # Save the model
    #model.save('saved_model.'+os.environ['SLURM_JOB_ID'])
    # model.save('saved_model.'+os.environ['SLURM_ARRAY_JOB_ID']+'.'+os.environ['SLURM_ARRAY_TASK_ID'])

    return model, test_dataset

In [ ]:
classes : 'CWLIntInput' = 10
precision : 'CWLIntInput' = tf.float32
epochs : 'CWLIntInput' = 1
batch_size : 'CWLIntInput' = 256
output_file : 'CWLFilePathOutput' = "./saved_model.keras"


In [6]:
trained_model, test_dataset = train_model(10, tf.float32, 1, 256)

170498071/170498071 [==============================] - 6s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                       

In [ ]:
trained_model.save(output_file)